In [50]:
import ast
import random
import matplotlib
from matplotlib import cm

In [1]:
def read_in_clusters(virus,subtype, mutation_type, exposure_cutoff, neighbor_cutoff):
    """
    Read in clusters of surface-exposed residues with fixations (or near fixations) that 
    were found by running saveClusters.py in pymol 
    """
    
    if subtype:
        with open(f'results/clusters/{virus}_{subtype}_{mutation_type}_{exposure_cutoff}_{neighbor_cutoff}_clusters.txt') as f:
            lines = f.readlines()
    else:
        with open(f'results/clusters/{virus}_{mutation_type}_{exposure_cutoff}_{neighbor_cutoff}_clusters.txt') as f:
            lines = f.readlines()
    
    clusters_dict = ast.literal_eval(lines[0])
    
    
    return clusters_dict

In [87]:
def write_ngl_file_epitopes(pdb_accession, virus, subtype, mutation_type, 
                            exposure_cutoff, neighbor_cutoff):
    """
    Write a ngl file to color structure by residues that are in clusters of surface residues with nonsyn fixations
    """
    
    cluster_dict = read_in_clusters(virus,subtype, mutation_type, exposure_cutoff, neighbor_cutoff)
    #restructure dict to have chain as key with dictionary as value
    # that dictionary will have key as cluster number and list of residues in that cluster as values
    chain_clusters_dict = {}
    for cluster_num, cluster_members in cluster_dict.items():
        for cluster_member in cluster_members:
            chain = cluster_member[0]
            res_num = cluster_member[1]
            
            if res_num.isnumeric():
                res_num_int = int(res_num)
            else:
                pass
            
            if chain not in chain_clusters_dict.keys():
                chain_clusters_dict[chain] = {cluster_num:[res_num_int]}
            else:
                if cluster_num not in chain_clusters_dict[chain].keys():
                    chain_clusters_dict[chain][cluster_num] = [res_num_int]
                else:
                    chain_clusters_dict[chain][cluster_num].append(res_num_int)
                
    #colors for separate clusters
    num_clusters = len(cluster_dict.keys())
    
    viridis = matplotlib.cm.get_cmap('viridis', num_clusters)
    rgba_colors = [viridis.colors[x] for x in range(num_clusters)] 
    hex_colors = [matplotlib.colors.to_hex(x)[1:] for x in rgba_colors]
    
    random.shuffle(hex_colors)

    #initialize list of text to write file encoding NGL structure for html
    text_lines = []
    
    #make color map for each chain
    chain_schemes = {}
    
    #all chains
    chains = []

    
    for chain, clusters in chain_clusters_dict.items():
        scheme_id = f'{chain}_epitopes_scheme'
        chain_schemes[chain] = scheme_id
        chains.append(chain)
        coloring_code_lines = [f"var {scheme_id} = NGL.ColormakerRegistry.addScheme(function (params) {{", 
                       "this.atomColor = function (atom) {", "if (atom.resno == 0) {return 0xf0f0f0}"]
        
        
        
            
        for cluster_num, cluster_members in clusters.items(): 
            coloring_code_lines.append(f"else if ({cluster_members}.includes(atom.resno)) {{return 0x{hex_colors[cluster_num-1]}}}")
#             coloring_code_lines.append(f"else if (atom.resno == {residue}) {{return 0x{hex_colors[cluster-1]}}}")

                    
        coloring_code_lines.append("else {return 0xf0f0f0}")
        coloring_code_lines.append(("}})"))
        coloring_code_lines.append(("\n"))

        text_lines += coloring_code_lines

        
    if subtype:
        ngl_filename = f'../../../atlas-of-viral-adaptation/{virus}_{subtype}/assets/{virus}_{subtype}_{mutation_type}_{exposure_cutoff}_{neighbor_cutoff}_epitopes_colorScheme.txt'
    else:
        ngl_filename = f'../../../atlas-of-viral-adaptation/{virus}/assets/{virus}_{mutation_type}_{exposure_cutoff}_{neighbor_cutoff}_epitopes_colorScheme.txt'
    with open(ngl_filename, 'w') as f:
        for line in text_lines:
            f.write(line)
            f.write('\n')
            

In [92]:
write_ngl_file_epitopes('4fnk', 'h3n2', None, 'nonsyn', 
                            15, 8)

In [93]:
write_ngl_file_epitopes('6u7h', '229e', None, 'nonsyn', 
                            15, 8)

In [94]:
write_ngl_file_epitopes('6ohw', 'oc43', 'a', 'nonsyn', 
                            15, 8)

In [95]:
write_ngl_file_epitopes('4nrj', 'vic', None, 'nonsyn', 
                            15, 8)

In [96]:
write_ngl_file_epitopes('4m4y', 'h1n1pdm', None, 'nonsyn', 
                            15, 8)